In [13]:
import mysql.connector as MySQL
import pandas as pd

In [1]:
host_name = "localhost"
username = 'root'
password = 'root' #dba password
db = "capstone"
table = 'web_raw_data'
auth_plugin = 'mysql_native_password'
conn = None

In [3]:
def connect_to_db(host_name, username, password, auth_plugin):
    try:
        conn = MySQL.connect(host=host_name, user = username, password = password, auth_plugin=auth_plugin)
    except MySQL.errorcode as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print('Connected!')
        return conn

In [38]:
conn=connect_to_db(host_name,username,password,auth_plugin)

Connected!


In [7]:
def execute_sql_command(conn,sql_query):
    try:
        c = conn.cursor()
        c.execute(sql_query)
        return c
    except Exception as err:
        print(err)

In [30]:
cursor=execute_sql_command(conn,"select url from {}.{} limit 10".format(db,table))

In [31]:
cursor.fetchall()

[('http://unemployment.ohio.gov/',),
 ('http://www.law.cornell.edu/wex/unemployment_compensation',),
 ('http://kiem-tv.com/?q=node/4395',),
 ('http://www.sciencedaily.com/releases/2012/09/120902113550.htm',),
 ('http://articles.timesofindia.indiatimes.com/2013-01-02/computing/36110957_1_electronic-cigarettes-traditional-cigarettes-regular-cigarettes',),
 ('http://www.ofspirit.com/brucefife1.htm',),
 ('http://www.ashtarcommandcrew.net/forum/topics/coconut-the-tree-of-life-superfood-cure-all',),
 ('http://www.mindbodygreen.com/0-4348/5-Ways-to-Use-Coconut-Oil.html',),
 ('http://www.co-optimus.com/review/1158/page/1/guardians-of-middle-earth-co-op-review.html',),
 ('http://katieholmes.com/',)]

In [32]:
cursor.close()

True

In [36]:
def last_sr_no(conn):
    cursor=execute_sql_command(conn,"select max(serial_number) from {}.{}".format(db,table))
    srno=cursor.fetchone()
    cursor.close()
    return srno[0]

In [39]:
last_sr_no(conn)

11382

In [27]:
feature_list = [{
        'commas':1, 'exclamations':0, 'dots':2, 'questions':1,
        'spelling_errors':104, 'text_complexity':1.937519, 'smog':3.0,
        'NN':55, 'VB':1, 'JJ':2, 'RB':0, 'DT':0, 'alexa_rank':4, 'document_url_y':"com"
    }]
df=pd.DataFrame(feature_list)

In [74]:
df['url']="https://www.google.com"

In [75]:
df

,commas,exclamations,dots,questions,spelling_errors,text_complexity,smog,NN,VB,JJ,RB,DT,alexa_rank,document_url_y,url
0,1,0,2,1,104,1.937519,3.0,55,1,2,0,0,4,com,https://www.google.com


In [78]:
"INSERT INTO {0}.{1} (`serial_number`,`{2}`) VALUES ({3},{4})".format(db,table,"`,`".join([str(i) for i in df.columns.tolist()]),11385,str(tuple(df.loc[0]))[1:-1])

"INSERT INTO capstone.web_raw_data (`serial_number`,`commas`,`exclamations`,`dots`,`questions`,`spelling_errors`,`text_complexity`,`smog`,`NN`,`VB`,`JJ`,`RB`,`DT`,`alexa_rank`,`document_url_y`,`url`) VALUES (11385,1, 0, 2, 1, 104, 1.937519, 3.0, 55, 1, 2, 0, 0, 4, 'com', 'https://www.google.com')"

In [79]:
def insertrow(conn,df):
    new_srno=last_sr_no(conn)+1
    cols = "`,`".join([str(i) for i in df.columns.tolist()])
    row=tuple(df.loc[0])
    sql = "INSERT INTO {0}.{1} (`serial_number`,`{2}`) VALUES ({3},{4})".format(db,table,cols,new_srno,str(row)[1:-1])
    cursor=conn.cursor()
    cursor.execute(sql)
    conn.commit()
    return new_srno

In [80]:
insertrow(conn,df)

11385

In [70]:
def updateseqcolumn(conn,seq_id,column_name,value):
    sql = "UPDATE {0}.{1} SET {2}=\"{3}\" WHERE SERIAL_NUMBER={4}".format(db,table,column_name,value,seq_id)
    cursor=conn.cursor()
    cursor.execute(sql)
    conn.commit()
    return seq_id

In [81]:
updateseqcolumn(conn,11385,"category","business")

11385

In [82]:
updateseqcolumn(conn,11385,"credibility_rating",3.54)

11385

In [135]:
class database:
    def __init__(self, host_name, username, password, auth_plugin,db,table) :
        self.host_name=host_name
        self.username=username
        self.password=password
        self.auth_plugin=auth_plugin
        self.db=db
        self.table=table

    def connect_to_db(self):
        try:
            conn = MySQL.connect(host=self.host_name, user = self.username, password = self.password, auth_plugin=self.auth_plugin)
        except Exception as err:
            print(err)
        else:
            print('Connected!')
            return conn

    def execute_query(self, sql_query):
        conn=self.connect_to_db()
        c = conn.cursor()
        c.execute(sql_query)
        return conn, c

    def last_sr_no(self):
        conn,cursor=self.execute_query("select max(serial_number) from {}.{}".format(self.db,self.table))
        srno=cursor.fetchone()
        conn.close()
        return srno[0]

    def insert_row(self,df):
        new_srno=self.last_sr_no()+1
        cols = "`,`".join([str(i) for i in df.columns.tolist()])
        row=tuple(df.loc[0])
        sql = "INSERT INTO {0}.{1} (`serial_number`,`{2}`) VALUES ({3},{4})".format(self.db,self.table,cols,new_srno,str(row)[1:-1])
        conn=self.connect_to_db()
        cursor=conn.cursor()
        cursor.execute(sql)
        conn.commit()
        conn.close()
        return new_srno

    def updateseqcolumn(self,seq_id,column_name,value):
        sql = "UPDATE {0}.{1} SET {2}=\"{3}\" WHERE SERIAL_NUMBER={4}".format(self.db,self.table,column_name,value,seq_id)
        conn=self.connect_to_db()
        cursor=conn.cursor()
        cursor.execute(sql)
        conn.commit()
        conn.close()
        return seq_id
    
    def delete_url(self,url):
        sql= "DELETE FROM {0}.{1} WHERE URL=\"{2}\"".format(self.db,self.table,url)
        conn=self.connect_to_db()
        cursor=conn.cursor()
        cursor.execute(sql)
        conn.commit()
        conn.close()
        
    def query_cols(self,seq_id,col_list):
        sql="select {0} from {1}.{2} where serial_number={3}".format(", ".join(col_list),db,table,seq_id)
        conn=self.connect_to_db()
        cursor=conn.cursor()
        cursor.execute(sql)
        result=cursor.fetchall()
        conn.commit()
        conn.close()
        return pd.DataFrame(result,columns=col_list)
        

In [136]:
db_util=database(host_name,username,password,auth_plugin,db,table)

In [110]:
db_util.last_sr_no()

Connected!


11385

In [111]:
df

,commas,exclamations,dots,questions,spelling_errors,text_complexity,smog,NN,VB,JJ,RB,DT,alexa_rank,document_url_y,url
0,1,0,2,1,104,1.937519,3.0,55,1,2,0,0,4,com,https://www.google.com


In [112]:
db_util.insert_row(df)

Connected!
Connected!


11386

In [113]:
db_util.updateseqcolumn(11386,"category","computer_internet")

Connected!


11386

In [114]:
db_util.updateseqcolumn(11386,"credibility_rating",3.56)

Connected!


11386

In [121]:
db_util.delete_url("https://www.google.com")

Connected!


In [138]:
db_util.query_cols(11080,['serial_number','url','category','credibility_rating'])

Connected!


,serial_number,url,category,credibility_rating
0,11080,http://www.erowid.org/library/books/seven_herb...,religion,4.500000000000000


In [1]:
host_name = "localhost"
username = 'root'
password = 'root' #dba password
db = "capstone"
table = 'web_raw_data'
auth_plugin = 'mysql_native_password'
conn = None

In [2]:
from utils.dbutils import database

In [3]:
dbutil=database(host_name,username,password,auth_plugin,db,table)

In [4]:
dbutil.reset_db()

Connected!
Connected!
Connected!


In [1]:
import json

In [3]:
cred_perf=json.load(open("models/credibility_performance.json","r"))

In [5]:
cred_perf['regTestAccuScore']

0.5